In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pharma-sales-data/5_model_building_removing_unnecessary_featureswithold_newcol.ipynb
/kaggle/input/pharma-sales-data/train_final.csv
/kaggle/input/pharma-sales-data/test_final.csv
/kaggle/input/kaggle-train/kaggle_train.csv
/kaggle/input/kaggle-train/kaggle_test.csv
/kaggle/input/test-footfall/test_footfall.csv


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
import json
from sklearn.neighbors import KNeighborsRegressor
import math
import os
from sklearn import tree
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

In [3]:
from sklearn.model_selection import validation_curve
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer

In [4]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [5]:
test = pd.read_csv('/kaggle/input/kaggle-train/kaggle_test.csv',sep=",", na_values=["?",",","#","NaN","unknown",""])
train=pd.read_csv('/kaggle/input/kaggle-train/kaggle_train.csv', sep=",", na_values=["?",",","#","NaN","unknown",""])

In [6]:
test_footfall=pd.read_csv('/kaggle/input/test-footfall/test_footfall.csv')

In [7]:
test["footfall_perday_permed"]=test_footfall

In [8]:
test.head()

,year,month,day,medicine,discounted,year_month,city_medicine,week,old_new,latitude,longitude,weekday,dayofweek,footfall_perday_permed
0,2018,7,1,1292,0,2018_07,1_1292,26,old,22.545412,88.356775,1,6,0.204360
1,2018,7,1,1,0,2018_07,1_1,26,old,22.545412,88.356775,1,6,0.175770
2,2018,7,1,2,1,2018_07,1_2,26,old,22.545412,88.356775,1,6,0.905007
3,2018,7,1,3,0,2018_07,1_3,26,old,22.545412,88.356775,1,6,0.765889
4,2018,7,1,4,0,2018_07,1_4,26,old,22.545412,88.356775,1,6,0.123034


In [9]:
train.head()

,year,month,day,medicine,sales,discounted,year_month,city_medicine,week,footfall_perday_permed,old_new,latitude,longitude,weekday,dayofweek
0,2017,1,2,1,28.0,1,2017_01,1_1,1,0.341523,old,22.545412,88.356775,0,0
1,2017,1,2,2,72.0,0,2017_01,1_2,1,0.878203,old,22.545412,88.356775,0,0
2,2017,1,2,3,128.0,0,2017_01,1_3,1,1.561250,old,22.545412,88.356775,0,0
3,2017,1,2,4,20.0,0,2017_01,1_4,1,0.243945,old,22.545412,88.356775,0,0
4,2017,1,2,5,4.0,0,2017_01,1_5,1,0.048789,old,22.545412,88.356775,0,0


In [10]:
catcols=["city_medicine","year_month","medicine","longitude","latitude","year","month","day","week","old_new"]
for col in catcols:
    train[col]=train[col].astype("category")
    test[col]=test[col].astype("category")

In [11]:
for col in ["discounted", "weekday", "dayofweek"]:
    train[col]=train[col].astype("int8")
    test[col]=test[col].astype("int8")

In [12]:
train.dtypes

year                      category
month                     category
day                       category
medicine                  category
sales                      float64
discounted                    int8
year_month                category
city_medicine             category
week                      category
footfall_perday_permed     float64
old_new                   category
latitude                  category
longitude                 category
weekday                       int8
dayofweek                     int8
dtype: object

In [13]:
X=train.drop(["sales"],axis=1)
y=train["sales"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.06,random_state=123,shuffle=False) 

In [15]:
X_train.tail(2)

,year,month,day,medicine,discounted,year_month,city_medicine,week,footfall_perday_permed,old_new,latitude,longitude,weekday,dayofweek
11016255,2018,6,1,1777,0,2018_06,9_1777,22,0.156365,old,23.021624,72.579707,0,4
11016256,2018,6,1,1778,0,2018_06,9_1778,22,0.039091,old,23.021624,72.579707,0,4


In [16]:
from category_encoders import TargetEncoder
tarcols=["city_medicine","year_month","medicine","longitude","latitude"]
encoder = TargetEncoder()
encoder.fit(X_train[tarcols], y_train)
X_train[tarcols]=encoder.transform(X_train[tarcols])
X_test[tarcols]=encoder.transform(X_test[tarcols])
test[tarcols]=encoder.transform(test[tarcols])

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
lecols=["year","month","day","week","old_new"]

for col in lecols:
    le.fit(X_train[col])
    X_train[col]=le.transform(X_train[col])
    X_test[col]=le.transform(X_test[col])
    test[col]=le.transform(test[col])
    

In [18]:
for col in ["year","month","day","week","old_new"]:
    X_train[col]=X_train[col].astype("int8")
    X_test[col]=X_test[col].astype("int8")
    test[col]=test[col].astype("int8")
    

In [19]:
scaler=StandardScaler()
scaler.fit(X_train["footfall_perday_permed"].reset_index())
X_train["footfall_perday_permed"]=scaler.transform(X_train["footfall_perday_permed"].reset_index())
X_test["footfall_perday_permed"]=scaler.transform(X_test["footfall_perday_permed"].reset_index())
test["footfall_perday_permed"]=scaler.transform(test["footfall_perday_permed"].reset_index())

In [20]:
X_train.dtypes

year                         int8
month                        int8
day                          int8
medicine                  float64
discounted                   int8
year_month                float64
city_medicine             float64
week                         int8
footfall_perday_permed    float64
old_new                      int8
latitude                  float64
longitude                 float64
weekday                      int8
dayofweek                    int8
dtype: object

In [21]:
from sklearn.linear_model import LinearRegression
reg= LinearRegression()
reg = reg.fit(X_train, y_train)
train_pred = reg.predict(X_train)
test_pred = reg.predict(X_test)

In [22]:
print("RMSE OF LINEAR REGRESSION=",np.sqrt(mean_squared_error(y_train, train_pred)))
print("MAPE OF LINEAR REGRESSION=",mean_absolute_percentage_error(y_train,train_pred))

RMSE OF LINEAR REGRESSION= 71.33559997876417
MAPE OF LINEAR REGRESSION= 91.11425829393839


In [23]:
print("RMSE OF LINEAR REGRESSION=",np.sqrt(mean_squared_error(y_test, test_pred)))
print("MAPE OF LINEAR REGRESSION=",mean_absolute_percentage_error(y_test,test_pred))

RMSE OF LINEAR REGRESSION= 57.296450819807724
MAPE OF LINEAR REGRESSION= 106.16650512486918


In [24]:
reg.predict(test)

array([370.00229455, 365.40854   , 448.35238682, ..., 199.41584493,
       224.34679697, 219.76826222])

In [25]:
xgb=XGBRegressor(random_state=1234)
xgb.fit(X_train, y_train)

[08:50:31] WARNING: ../src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=1234,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [26]:
train_XGB_pred = xgb.predict(X_train)
test_XGB_pred = xgb.predict(X_test)

In [27]:
print("RMSE OF XGB TRAIN REGRESSION=",np.sqrt(mean_squared_error(y_train, train_XGB_pred)))
print("MAPE OF XGB TRAIN REGRESSION=",mean_absolute_percentage_error(y_train,train_XGB_pred))

RMSE OF XGB TRAIN REGRESSION= 74.10938005828896
MAPE OF XGB TRAIN REGRESSION= 76.38180055333126


In [28]:
print("RMSE OF XGB TEST REGRESSION=",np.sqrt(mean_squared_error(y_test, test_XGB_pred)))
print("MAPE OF XGB TEST REGRESSION=",mean_absolute_percentage_error(y_test,test_XGB_pred))

RMSE OF XGB TEST REGRESSION= 63.10179262078612
MAPE OF XGB TEST REGRESSION= 89.05701645387916


In [29]:
test=test[['year', 'month', 'day', 'medicine', 'discounted', 'year_month', 'city_medicine', 'week', 'footfall_perday_permed', 'old_new', 'latitude', 'longitude', 'weekday', 'dayofweek']]

In [30]:
XGB_pred = xgb.predict(test)

In [31]:
GBM=GradientBoostingRegressor(random_state=0)
GBM = GBM.fit(X_train, y_train)
train_GBM_pred = GBM.predict(X_train)
test_GBM_pred = GBM.predict(X_test)

In [32]:
print("RMSE OF GBM TRAIN REGRESSION=",np.sqrt(mean_squared_error(y_train, train_GBM_pred)))
print("MAPE OF GBM TRAIN REGRESSION=",mean_absolute_percentage_error(y_train,train_GBM_pred))

RMSE OF GBM TRAIN REGRESSION= 68.06524845684257
MAPE OF GBM TRAIN REGRESSION= 81.00052483413613


In [33]:
print("RMSE OF GBM TEST REGRESSION=",np.sqrt(mean_squared_error(y_test, test_GBM_pred)))
print("MAPE OF GBM TEST REGRESSION=",mean_absolute_percentage_error(y_test,test_GBM_pred))

RMSE OF GBM TEST REGRESSION= 55.032030396057095
MAPE OF GBM TEST REGRESSION= 100.02133037831531


In [37]:
testp=pd.read_csv('/kaggle/input/pharma-sales-data/test_final.csv',sep=",", na_values=["?",",","#","NaN","unknown",""])
testp=testp[["id"]]
testp["sales"]=XGB_pred

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64
# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = testp.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
#df = pd.DataFrame(np.random.randn(50, 4), columns=list('ABCD'))

# create a link to download the dataframe
create_download_link(testp)

This is our best model as of now which is giving rmse of 62 on SCT which is close to test data rmse of XGB model 63.101 